In [3]:
import sys
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label

# Add your custom library path
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/")

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
current_script_dir = os.path.dirname(os.path.abspath(__file__))

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
# Assuming get_trials_with_outlier_analysis is in your general_utils
from src.analysis.utils.general_utils import get_trials_with_outlier_analysis

def outlier_analysis(subjects, event, times, outlier_threshold, create_outlier_plots, mark_outliers_as_nan, channels=None, task='GlobalLocal', LAB_root=None):
    """
    Performs outlier analysis for a list of subjects.
    """
    # Determine LAB_root once, outside the loop if it's not provided.
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    for sub in subjects:
        print(f'-----\nProcessing subject: {sub}')

        # The save_dir is specific to each subject, so it should be defined inside the loop
        save_dir = os.path.join(LAB_root, 'derivatives', 'tests', 'outlierAnalysis', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Load data for the subject
        layout = get_data(task, root=LAB_root)
        filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                               extension='.edf', desc='clean', preload=False)

        good = crop_empty_data(filt)
        
        print(f"Channels before dropping bads: {len(good.ch_names)}")
        good.info['bads'] = channel_outlier_marker(good, 3, 2)
        print("Marked bad channels:", good.info['bads'])
        good.drop_channels(good.info['bads'])
        print(f"Channels after dropping bads: {len(good.ch_names)}")

        good.load_data()

        # Use a local variable for the channels to be picked for the current subject.
        channels_to_use = channels
        if channels_to_use is not None:
            # Validate the provided channels for the current subject
            invalid_channels = [ch for ch in channels_to_use if ch not in good.ch_names]
            if invalid_channels:
                raise ValueError(f"Subject {sub}: The following channels are not valid: {invalid_channels}")
            good.pick_channels(channels_to_use)

        # Re-referencing
        ch_type = good.get_channel_types(only_data_chs=True)[0]
        good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

        # Run the analysis and plotting
        trials = get_trials_with_outlier_analysis(
            data=good, 
            events=event, 
            times=times, 
            outlier_threshold=outlier_threshold, 
            create_outlier_plots=create_outlier_plots, 
            mark_outliers_as_nan=mark_outliers_as_nan, 
            save_dir=save_dir
        )
        print(f"Finished processing for {sub}. Plots saved in {save_dir}")

# ===================================================================
# Script Execution Starts Here
# ===================================================================

# Define all parameters for the analysis
# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
subjects = ['D0057']
event_type = ['Stimulus']
time_window = (-1, 1.5)
outlier_threshold = 10
create_outlier_plots = True
mark_outliers_as_nan = True
task = 'GlobalLocal'

# Call the function with the correct, simplified set of arguments.
outlier_analysis(
    subjects=subjects_list, 
    event=event_type, 
    times=time_window, 
    outlier_threshold=outlier_threshold, 
    create_outlier_plots=create_outlier_plots, 
    mark_outliers_as_nan=mark_outliers_as_nan, 
    task=task
)

print("\nAll subjects processed!")

NameError: name '__file__' is not defined